# Generate index weights using dedicated service - Python

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index weights sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns index components and weights.

### Services used
This sample uses *gRPC requests* in order to retrieve index components and weights from the hosted service. The queried endpoint in this script are:
* *ComponentsService*: to directly retrieve index data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.indices.v1*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run symbology queries

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas

In [ ]:
pip install -i https://test.pypi.org/simple/ systemathics.apis==0.9.27

In [ ]:
import os
import grpc
import pandas as pd
import google.protobuf as pb
import systemathics.apis.services.indices.v1.components_pb2 as indices
import systemathics.apis.services.indices.v1.components_pb2_grpc as indices_service

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request

The following code snippet enables to select the **index** by its *Name*:

In [ ]:
index = 'nasdaq 100'

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
# generate the indices request
request = indices.ComponentsRequest( 
    identifier = index
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the indices service
    service = indices_service.ComponentsServiceStub(channel)
    
    # process the indices request
    response = service.Components(
        request = request, 
        metadata = [('authorization', token)]
    )

### Step 4: Visualize index data

#### 4.1 Retrieve components and weights

In [ ]:
# create the pandas dataframe for indices' components and weights
tickers = [c.identifier.ticker for c in response.components]
exchanges =[c.identifier.exchange for c in response.components]
weights = [c.weight for c in response.components]

d = {'Ticker': tickers, 'Exchange': exchanges, 'Weight': weights }
df = pd.DataFrame(data=d)
df

#### 4.2 Plot components and weights

In [ ]:
import plotly.express as px

fig = px.pie(df, values='Weight', names='Ticker')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()